In [8]:
# Simulates basic flip-flop circuits in Python using a functional approach.
# Implements SR Latch and D Flip-Flop using basic logic gate functions (NOT, AND, OR, NOR).
# All inputs are assumed to be 0 or 1 (treated as boolean).
# State is managed using a dictionary instead of class instances.

# Basic gate functions
def not_gate(a):
    return 1 if a == 0 else 0

def and_gate(a, b):
    return 1 if a == 1 and b == 1 else 0

def or_gate(a, b):
    return 1 if a == 1 or b == 1 else 0

def nor_gate(a, b):
    return 1 if a == 0 and b == 0 else 0

# SR Latch: Basic memory element using NOR gates
def sr_latch(S, R, state):
    # state is a dictionary with 'Q' and 'Q_bar' keys
    Q = state.get('Q', 0)  # Default initial Q = 0
    Q_bar = state.get('Q_bar', 1)  # Default initial Q_bar = 1
    # NOR latch: Q = NOR(R, Q_bar), Q_bar = NOR(S, Q)
    next_Q = nor_gate(R, Q_bar)
    next_Q_bar = nor_gate(S, next_Q)
    # Update state
    state['Q'] = next_Q
    state['Q_bar'] = next_Q_bar
    return next_Q, next_Q_bar, state

# D Flip-Flop: Edge-triggered memory element
def d_flip_flop(D, clock, state):
    # state is a dictionary with 'Q', 'Q_bar', and 'prev_clock' keys
    prev_clock = state.get('prev_clock', 0)  # Track previous clock for edge detection
    Q = state.get('Q', 0)  # Default initial Q = 0
    Q_bar = state.get('Q_bar', 1)  # Default initial Q_bar = 1
    
    # Detect rising edge (clock transitions from 0 to 1)
    clock_edge = and_gate(clock, not_gate(prev_clock))
    
    # Generate S and R for SR Latch
    S = and_gate(D, clock_edge)
    R = and_gate(not_gate(D), clock_edge)
    
    # Update SR Latch
    next_Q, next_Q_bar, latch_state = sr_latch(S, R, {'Q': Q, 'Q_bar': Q_bar})
    
    # Update state with new values and current clock
    state['Q'] = next_Q
    state['Q_bar'] = next_Q_bar
    state['prev_clock'] = clock
    
    return next_Q, next_Q_bar, state

# Example usage
if __name__ == "__main__":
    # Initialize state dictionaries
    sr_state = {'Q': 0, 'Q_bar': 1}
    dff_state = {'Q': 0, 'Q_bar': 1, 'prev_clock': 0}
    
    # Test SR Latch
    print("SR Latch: S=0, R=0 -> Q, Q_bar =", sr_latch(0, 0, sr_state)[:2])  # Hold state
    print("SR Latch: S=1, R=0 -> Q, Q_bar =", sr_latch(1, 0, sr_state)[:2])  # Set (Q=1)
    print("SR Latch: S=0, R=1 -> Q, Q_bar =", sr_latch(0, 1, sr_state)[:2])  # Reset (Q=0)
    print("SR Latch: S=0, R=0 -> Q, Q_bar =", sr_latch(0, 0, sr_state)[:2])  # Hold state
    
    # Test D Flip-Flop
    print("\nD Flip-Flop: D=1, Clock=0 -> Q, Q_bar =", d_flip_flop(1, 0, dff_state)[:2])  # No change
    print("D Flip-Flop: D=1, Clock=1 -> Q, Q_bar =", d_flip_flop(1, 1, dff_state)[:2])  # Rising edge, Q=1
    print("D Flip-Flop: D=0, Clock=0 -> Q, Q_bar =", d_flip_flop(0, 0, dff_state)[:2])  # No change
    print("D Flip-Flop: D=0, Clock=1 -> Q, Q_bar =", d_flip_flop(0, 1, dff_state)[:2])  # Rising edge, Q=0

SR Latch: S=0, R=0 -> Q, Q_bar = (0, 1)
SR Latch: S=1, R=0 -> Q, Q_bar = (0, 0)
SR Latch: S=0, R=1 -> Q, Q_bar = (0, 1)
SR Latch: S=0, R=0 -> Q, Q_bar = (0, 1)

D Flip-Flop: D=1, Clock=0 -> Q, Q_bar = (0, 1)
D Flip-Flop: D=1, Clock=1 -> Q, Q_bar = (0, 0)
D Flip-Flop: D=0, Clock=0 -> Q, Q_bar = (1, 0)
D Flip-Flop: D=0, Clock=1 -> Q, Q_bar = (0, 1)


In [ ]:
# Simulates a D Flip-Flop circuit and visualizes its signals like an oscilloscope using matplotlib.
# Implements basic logic gates and D Flip-Flop functionally.
# All inputs are assumed to be 0 or 1 (treated as boolean).

import matplotlib.pyplot as plt
import numpy as np

# Basic gate functions
def not_gate(a):
    return 1 if a == 0 else 0

def and_gate(a, b):
    return 1 if a == 1 and b == 1 else 0

def or_gate(a, b):
    return 1 if a == 1 or b == 1 else 0

def nor_gate(a, b):
    return 1 if a == 0 and b == 0 else 0

# SR Latch: Basic memory element using NOR gates
def sr_latch(S, R, state):
    Q = state.get('Q', 0)  # Default initial Q = 0
    Q_bar = state.get('Q_bar', 1)  # Default initial Q_bar = 1
    next_Q = nor_gate(R, Q_bar)
    next_Q_bar = nor_gate(S, next_Q)
    state['Q'] = next_Q
    state['Q_bar'] = next_Q_bar
    return next_Q, next_Q_bar, state

# D Flip-Flop: Edge-triggered memory element
def d_flip_flop(D, clock, state):
    prev_clock = state.get('prev_clock', 0)
    Q = state.get('Q', 0)
    Q_bar = state.get('Q_bar', 1)
    clock_edge = and_gate(clock, not_gate(prev_clock))
    S = and_gate(D, clock_edge)
    R = and_gate(not_gate(D), clock_edge)
    next_Q, next_Q_bar, latch_state = sr_latch(S, R, {'Q': Q, 'Q_bar': Q_bar})
    state['Q'] = next_Q
    state['Q_bar'] = next_Q_bar
    state['prev_clock'] = clock
    return next_Q, next_Q_bar, state

# Simulate D Flip-Flop over time with a sequence of inputs
def simulate_d_flip_flop(inputs):
    state = {'Q': 0, 'Q_bar': 1, 'prev_clock': 0}
    time_steps = []
    D_values = []
    clock_values = []
    Q_values = []
    Q_bar_values = []
    
    for t, (D, clock) in enumerate(inputs):
        Q, Q_bar, state = d_flip_flop(D, clock, state)
        time_steps.append(t)
        D_values.append(D)
        clock_values.append(clock)
        Q_values.append(Q)
        Q_bar_values.append(Q_bar)
    
    return time_steps, D_values, clock_values, Q_values, Q_bar_values

# Visualize signals like an oscilloscope
def plot_oscilloscope(time_steps, D_values, clock_values, Q_values, Q_bar_values):
    plt.figure(figsize=(10, 8))
    
    # Plot D signal
    plt.subplot(4, 1, 1)
    plt.step(time_steps, D_values, where='post', color='blue')
    plt.title('D Flip-Flop Oscilloscope Simulation')
    plt.ylabel('D')
    plt.ylim(-0.2, 1.2)
    plt.grid(True)
    
    # Plot Clock signal
    plt.subplot(4, 1, 2)
    plt.step(time_steps, clock_values, where='post', color='green')
    plt.ylabel('Clock')
    plt.ylim(-0.2, 1.2)
    plt.grid(True)
    
    # Plot Q signal
    plt.subplot(4, 1, 3)
    plt.step(time_steps, Q_values, where='post', color='red')
    plt.ylabel('Q')
    plt.ylim(-0.2, 1.2)
    plt.grid(True)
    
    # Plot Q_bar signal
    plt.subplot(4, 1, 4)
    plt.step(time_steps, Q_bar_values, where='post', color='purple')
    plt.ylabel('Q_bar')
    plt.xlabel('Time Step')
    plt.ylim(-0.2, 1.2)
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    # Define input sequence: (D, Clock) pairs
    inputs = [
        (0, 0),  # t=0
        (1, 0),  # t=1
        (1, 1),  # t=2 (rising edge, Q should become 1)
        (1, 1),  # t=3
        (1, 0),  # t=4
        (0, 0),  # t=5
        (0, 1),  # t=6 (rising edge, Q should become 0)
        (0, 1),  # t=7
        (1, 0),  # t=8
        (1, 1),  # t=9 (rising edge, Q should become 1)
    ]
    
    # Simulate and visualize
    time_steps, D_values, clock_values, Q_values, Q_bar_values = simulate_d_flip_flop(inputs)
    plot_oscilloscope(time_steps, D_values, clock_values, Q_values, Q_bar_values)